In [0]:
from google.colab import files
from google.colab import drive
# Install Kaggle library
!pip install -q kaggle
import bz2
import pandas as pd
import string
import re

# The dataset used in the experiements is **Amazon Reviews for Sentiment Analysis** "https://www.kaggle.com/bittlingmayer/amazonreviews"  

This notebook explains the steps to pull the data from kaggle and perform basic cleaning for future work



This step is specific for google colab and will not required if it is experiemented in local or cloud machine

Mount the gdrive by entering the authorization code

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Create the folder NLP-Basic-to-Bert and change your direction to NLP-Basic-to-Bert

In [0]:
mkdir '/content/gdrive/My Drive/Colab Notebooks/NLP-Basic-to-Bert'

In [14]:
cd '/content/gdrive/My Drive/Colab Notebooks/NLP-Basic-to-Bert'

/content/gdrive/My Drive/Colab Notebooks/NLP-Basic-to-Bert


Follow the instruction from the blog and download the kaggle.json 
https://towardsdatascience.com/setting-up-kaggle-in-google-colab-ebb281b61463

In [16]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
#Create the folder ~/.kaggle and copy the kaggle.json to the given folder
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

In [22]:
#Download the amazon reviews from kaggle data set
!kaggle datasets download -d bittlingmayer/amazonreviews

amazonreviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
#Unzip the amazonreviews 
!unzip amazonreviews.zip

Archive:  amazonreviews.zip
replace test.ft.txt.bz2? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
# Function to seperate labels and text
def get_labels_and_texts(file):
    
    text_data = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        text_data.append([x[10:].strip(),int(x[9]) - 1])
    return text_data

In [0]:
# Convert the data into csv
train_data = get_labels_and_texts('train.ft.txt.bz2')
train_df = pd.DataFrame(train_data,columns=['user_review','user_rating'])
del train_data
test_data = get_labels_and_texts('test.ft.txt.bz2')
test_df = pd.DataFrame(test_data,columns=['user_review','user_rating'])
del test_data

In [31]:
train_df.head(10)

,user_review,user_rating
0,Stuning even for the non-gamer: This sound tra...,1
1,The best soundtrack ever to anything.: I'm rea...,1
2,Amazing!: This soundtrack is my favorite music...,1
3,Excellent Soundtrack: I truly like this soundt...,1
4,"Remember, Pull Your Jaw Off The Floor After He...",1
5,an absolute masterpiece: I am quite sure any o...,1
6,"Buyer beware: This is a self-published book, a...",0
7,Glorious story: I loved Whisper of the wicked ...,1
8,A FIVE STAR BOOK: I just finished reading Whis...,1
9,Whispers of the Wicked Saints: This was a easy...,1


In [0]:
# Remove the punctuations
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

train_df["user_review"] = train_df["user_review"].apply(lambda text: remove_punctuation(text))
test_df["user_review"] = test_df["user_review"].apply(lambda text: remove_punctuation(text))

In [0]:
# Remove the urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
train_df["user_review"] = train_df["user_review"].apply(lambda text: remove_urls(text))
test_df["user_review"] = test_df["user_review"].apply(lambda text: remove_urls(text))

In [38]:
# Data after cleaning up
train_df.head(10)

,user_review,user_rating
0,Stuning even for the nongamer This sound track...,1
1,The best soundtrack ever to anything Im readin...,1
2,Amazing This soundtrack is my favorite music o...,1
3,Excellent Soundtrack I truly like this soundtr...,1
4,Remember Pull Your Jaw Off The Floor After Hea...,1
5,an absolute masterpiece I am quite sure any of...,1
6,Buyer beware This is a selfpublished book and ...,0
7,Glorious story I loved Whisper of the wicked s...,1
8,A FIVE STAR BOOK I just finished reading Whisp...,1
9,Whispers of the Wicked Saints This was a easy ...,1


In [39]:
print("The number of training and testing data is",len(train_df),len(test_df))

The number of training and testing data is 3600000 400000


# Training data set is reduced to 5 lakhs due to constrains in resources. Full training set can be used if resources are not an issue

In [0]:
train_sample = train_df.head(500000)

In [0]:
train_sample.to_csv('training_data.csv',index=False)
test_df.to_csv('testing_data.csv',index=False)